In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

data_path = Path('/recsys_data/RecSys/h_and_m_personalized_fashion_recommendation/')
res_dir = "/recsys_data/RecSys/TransformerRec"

In [2]:
transactions = pd.read_csv(
    data_path / 'transactions_train.csv',
    # set dtype or pandas will drop the leading '0' and convert to int
    dtype={'article_id': str} 
)

submission = pd.read_csv(data_path / 'sample_submission.csv')

In [3]:
print(transactions.shape)
transactions.head()

(31788324, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [4]:
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])
transactions['t_dat'].min(), transactions['t_dat'].max()

(Timestamp('2018-09-20 00:00:00'), Timestamp('2020-09-22 00:00:00'))

In [5]:
transactions_5w = transactions[transactions['t_dat'] >= pd.to_datetime('2020-07-15')].copy()  # last 5 weeks
transactions_3w = transactions[transactions['t_dat'] >= pd.to_datetime('2020-08-31')].copy()  # last 3 weeks
transactions_2w = transactions[transactions['t_dat'] >= pd.to_datetime('2020-09-07')].copy()  # last 2 weeks
transactions_1w = transactions[transactions['t_dat'] >= pd.to_datetime('2020-09-15')].copy()  # one week before

In [6]:
purchase_dict_5w = {}

for i,x in enumerate(zip(transactions_5w['customer_id'], transactions_5w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_5w:
        purchase_dict_5w[cust_id] = {}
    
    if art_id not in purchase_dict_5w[cust_id]:
        purchase_dict_5w[cust_id][art_id] = 0
    
    purchase_dict_5w[cust_id][art_id] += 1
    
print(len(purchase_dict_5w))

dummy_list_5w = list((transactions_5w['article_id'].value_counts()).index)[:12]

437365


In [7]:
purchase_dict_3w = {}

for i,x in enumerate(zip(transactions_3w['customer_id'], transactions_3w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_3w:
        purchase_dict_3w[cust_id] = {}
    
    if art_id not in purchase_dict_3w[cust_id]:
        purchase_dict_3w[cust_id][art_id] = 0
    
    purchase_dict_3w[cust_id][art_id] += 1
    
print(len(purchase_dict_3w))

dummy_list_3w = list((transactions_3w['article_id'].value_counts()).index)[:12]

196319


In [8]:
purchase_dict_2w = {}

for i,x in enumerate(zip(transactions_2w['customer_id'], transactions_2w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_2w:
        purchase_dict_2w[cust_id] = {}
    
    if art_id not in purchase_dict_2w[cust_id]:
        purchase_dict_2w[cust_id][art_id] = 0
    
    purchase_dict_2w[cust_id][art_id] += 1
    
print(len(purchase_dict_2w))

dummy_list_2w = list((transactions_2w['article_id'].value_counts()).index)[:12]

143455


In [9]:
purchase_dict_1w = {}

for i,x in enumerate(zip(transactions_1w['customer_id'], transactions_1w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_1w:
        purchase_dict_1w[cust_id] = {}
    
    if art_id not in purchase_dict_1w[cust_id]:
        purchase_dict_1w[cust_id][art_id] = 0
    
    purchase_dict_1w[cust_id][art_id] += 1
    
print(len(purchase_dict_1w))

dummy_list_1w = list((transactions_1w['article_id'].value_counts()).index)[:12]

75481


In [10]:
print(submission.shape)
submission.head()

(1371980, 2)


,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


## Modify Default Prediction
    - based on customer segment

In [25]:
df_cust = pd.read_csv(data_path/ "customers.csv")
df_cust['customer_id']=df_cust['customer_id'].astype(str)
df_cust['age_bracket'] = pd.cut(df_cust.age, [0,20,30,40,50,60,70,100], include_lowest=True)
df_cust['age_bracket'].value_counts()

(20.0, 30.0]      512397
(40.0, 50.0]      216880
(30.0, 40.0]      210991
(50.0, 60.0]      209856
(-0.001, 20.0]    126779
(60.0, 70.0]       63363
(70.0, 100.0]      15853
Name: age_bracket, dtype: int64

In [70]:
transactions_1w['customer_id'] = transactions_1w['customer_id'].astype(str)
transactions_1w = transactions_1w.merge(df_cust, on="customer_id", how="left")
transactions_1w['age_bracket'].value_counts()

(20.0, 30.0]      116190
(30.0, 40.0]       43322
(40.0, 50.0]       39755
(50.0, 60.0]       37898
(-0.001, 20.0]     17499
(60.0, 70.0]        9032
(70.0, 100.0]       1676
Name: age_bracket, dtype: int64

In [71]:
transactions_2w = transactions_2w.merge(df_cust, on="customer_id", how="left")
transactions_2w['age_bracket'].value_counts()

(20.0, 30.0]      250204
(30.0, 40.0]       93007
(40.0, 50.0]       85128
(50.0, 60.0]       79783
(-0.001, 20.0]     36904
(60.0, 70.0]       19150
(70.0, 100.0]       3630
Name: age_bracket, dtype: int64

In [72]:
transactions_3w = transactions_3w.merge(df_cust, on="customer_id", how="left")
transactions_3w['age_bracket'].value_counts()

(20.0, 30.0]      363060
(30.0, 40.0]      137542
(40.0, 50.0]      127740
(50.0, 60.0]      116807
(-0.001, 20.0]     55033
(60.0, 70.0]       27899
(70.0, 100.0]       5265
Name: age_bracket, dtype: int64

In [76]:
transactions_1w.shape

(266364, 12)

In [77]:
def get_default(x):
    return list((x.value_counts()).index)[:12]

# dummy_dict = tr_temp.groupby("age_bracket")['article_id'].apply(lambda x: get_default(x)).T.to_dict()
dummy_1w = pd.DataFrame(transactions_1w.groupby("age_bracket")['article_id'].apply(lambda x: get_default(x))).reset_index()
dummy_dict_1w = df_cust[['customer_id', 'age_bracket']].merge(dummy_1w, on="age_bracket", how="right")[['customer_id', 'article_id']].set_index("customer_id")

dummy_2w = pd.DataFrame(transactions_2w.groupby("age_bracket")['article_id'].apply(lambda x: get_default(x))).reset_index()
dummy_dict_2w = df_cust[['customer_id', 'age_bracket']].merge(dummy_2w, on="age_bracket", how="right")[['customer_id', 'article_id']].set_index("customer_id")

dummy_3w = pd.DataFrame(transactions_3w.groupby("age_bracket")['article_id'].apply(lambda x: get_default(x))).reset_index()
dummy_dict_3w = df_cust[['customer_id', 'age_bracket']].merge(dummy_3w, on="age_bracket", how="right")[['customer_id', 'article_id']].set_index("customer_id")


In [78]:
dummy_dict_1w.shape

(1356119, 1)

In [66]:
not_so_fancy_but_fast_benchmark = submission[['customer_id']]
prediction_list = []

dummy_list = list((transactions_1w['article_id'].value_counts()).index)[:12]
dummy_pred = ' '.join(dummy_list)
count_ids = 0

for i, cust_id in enumerate(submission['customer_id'].values.reshape((-1,))):
    if cust_id in purchase_dict_1w:
        l = sorted((purchase_dict_1w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+dummy_list_1w[:(12-len(l))])
    elif cust_id in purchase_dict_2w:
        l = sorted((purchase_dict_2w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+dummy_list_2w[:(12-len(l))])
    elif cust_id in purchase_dict_3w:
        l = sorted((purchase_dict_3w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+dummy_list_3w[:(12-len(l))])
    elif cust_id in purchase_dict_5w:
        l = sorted((purchase_dict_5w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+dummy_list_5w[:(12-len(l))])
    else:
#         s = dummy_pred
        if str(dummy_dict.loc[cust_id]['article_id']) == 'nan':
            s = dummy_pred
        else:
            s = ' '.join(dummy_dict.loc[cust_id]['article_id'])
        count_ids += 1
    prediction_list.append(s)

not_so_fancy_but_fast_benchmark['prediction'] = prediction_list
print(not_so_fancy_but_fast_benchmark.shape)
print(count_ids, "customers not in the last 3 weeks")
not_so_fancy_but_fast_benchmark.head()

(1371980, 2)
934615 customers not in the last 3 weeks


,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0751471001 0909370001 0915526001 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0924243001 0866731001 0909370001 0923758001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0924243001 0924243002 0923758001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0924243001 0928206001 0930380001 0924243002 09...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0896152002 0730683050 0927530004 0791587015 07...


In [67]:
not_so_fancy_but_fast_benchmark.to_csv(os.path.join(res_dir, "submission_nsfbfb4.txt"), header=True, index=False)

In [68]:
! kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f /recsys_data/RecSys/TransformerRec/submission_nsfbfb4.txt -m "attempt-16"

100%|████████████████████████████████████████| 258M/258M [00:07<00:00, 37.4MB/s]
Successfully submitted to H&M Personalized Fashion Recommendations

Score = 0.020 (highest so far); does not improve after adding another period

In [20]:
len(set(purchase_dict_1w.keys()) | set(purchase_dict_2w.keys()) | set(purchase_dict_3w.keys()))

196319

In [22]:
dummy_pred

'0924243001 0924243002 0923758001 0918522001 0909370001 0866731001 0751471001 0915529003 0915529005 0448509014 0762846027 0714790020'

In [11]:
1371980 - 1175661

196319

In [69]:
df_cust

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,age_bracket
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,"(40.0, 50.0]"
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...,"(20.0, 30.0]"
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,"(20.0, 30.0]"
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...,"(50.0, 60.0]"
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...,"(50.0, 60.0]"
...,...,...,...,...,...,...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,NaN,NaN,ACTIVE,NONE,24.0,7aa399f7e669990daba2d92c577b52237380662f36480b...,"(20.0, 30.0]"
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,NaN,NaN,ACTIVE,NONE,21.0,3f47f1279beb72215f4de557d950e0bfa73789d24acb5e...,"(20.0, 30.0]"
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,1.0,1.0,ACTIVE,Regularly,21.0,4563fc79215672cd6a863f2b4bf56b8f898f2d96ed590e...,"(20.0, 30.0]"
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,1.0,1.0,ACTIVE,Regularly,18.0,8892c18e9bc3dca6aa4000cb8094fc4b51ee8db2ed14d7...,"(-0.001, 20.0]"


In [79]:
df_cust['Active'].value_counts()

1.0    464404
Name: Active, dtype: int64

In [80]:
df_cust['club_member_status'].value_counts()

ACTIVE        1272491
PRE-CREATE      92960
LEFT CLUB         467
Name: club_member_status, dtype: int64

In [81]:
df_cust['fashion_news_frequency'].value_counts()

NONE         877711
Regularly    477416
Monthly         842
None              2
Name: fashion_news_frequency, dtype: int64